In [ ]:
# This notebook will collect the necessary data 
# warning: ensure you put in a nice delay between queries as a courtesy to the server 
# as well as to not get kick off 

# author: marieke.van.erp@dh.huc.knaw.nl 


To gather the information for the mapping, perform the following 3 steps:  

1. Run the following sparql queries on https://qlever.cs.uni-freiburg.de/wikidata

PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?entity ?entityLabel ?property ?object 
WHERE
  {
  VALUES ?type { wd:Q43229 } . 
  ?entity wdt:P31*/wdt:P279* ?type .
  ?entity ?property ?object . 
  ?entity rdfs:label ?entityLabel .
  FILTER (LANG(?entityLabel) = "en") .
}

2. Download the results as tsv 

3. In a terminal, run the following command on the file to obtain a list of unique properties for the mapping: 
cut -f3 < <FILENAME> | sort | uniq > <UNIQUE_PROPERTIES> 

The properties were analysed manually, the resulting mapping file can be found in data/ and is further described in Section 5 of the paper. 

To obtain the data for the company, enterprise and business analyses (Section 6 in the paper): 
1. Run the following sparql queries on https://query.wikidata.org/ to get the business entities: 

SELECT ?entity ?entityLabel
WHERE
{ 
  ?entity wdt:P31 wd:Q4830453 . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}

2. Save the results as businesses.tsv without headers 

3. Run the following sparql queries on https://query.wikidata.org/ to get the enterprise entities: 
SELECT ?entity ?entityLabel
WHERE
{ 
  ?entity wdt:P31 wd:Q6881511 . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}

4. Save the results as enterprises.tsv without headers 

5. Run the following sparql queries on https://query.wikidata.org/ to get the company entities: 
SELECT ?entity ?entityLabel
WHERE
{ 
  ?entity wdt:P31 wd:Q783794 . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}

6. Save the results as companies.tsv without headers 

Note: a combined query such as the following should return the same results, but probably results in a timeout: 
   SELECT ?entity ?entityLabel
   WHERE
    { 
    VALUES ?type { wd:Q4830453 wd:Q6881511 wd:Q783794 } .  
    ?entity wdt:P31 ?type . 
    } 
    
7. Combine the three files to get a list of unique ids and labels. For example by the following command: 
cat businesses.tsv enterprises.tsv companies.tsv >> businesses-enterprises-companies.tsv 

8. Ensure the file is in the directory data/

In [ ]:
# Preamble: load the necessary modules 
import pandas as pd
import pywikibot
import glob 
import time
import datetime 

In [ ]:
# Preamble: tell the pywikibot which database to query: 
site = pywikibot.Site('wikidata', 'wikidata')

In [ ]:
# Preamble: function that reads in an entity id, queries Wikidata for a set of properties about that entity 
# and how to store the information 
def getchangeinfo(site, item_qid, claims_of_interest):
    item = pywikibot.ItemPage(site, item_qid)
    info = []
    
    try:
        item_dict = item.get() #Get the item dictionary
    
        item_name = "DUMMY-no-en-label-DUMMY"
    
        if "en" in item_dict["labels"]:
            item_name = item_dict["labels"]["en"]
    
        clm_dict = item_dict["claims"] # Get the claim dictionary
        
        for coi in claims_of_interest:
            #print("Processing: " + str(coi))
            claim_lookup = coi[0]
            claim_lookup_name = coi[1]
            start_time = None
            end_time = None
            point_in_time  = None
            try:
                claims = clm_dict[claim_lookup]
                for c in claims:
                    #print("yay")
                    clm_trgt = c.getTarget()
                    targetid = None
                    targetstr = str(clm_trgt)
                    if type(clm_trgt) is pywikibot.WbMonolingualText:
                        targetstr = clm_trgt.text
                    if type(clm_trgt) is pywikibot.ItemPage:
                        targetstr = clm_trgt.labels['en']
                        targetid = clm_trgt.title()
                    if type(clm_trgt) is pywikibot.WbQuantity:
                        targetstr = clm_trgt.amount
                    else:
                        pass
                    if c.qualifiers:
                        for qualifier_property, qualifiers in c.qualifiers.items():
                            for qualifier in qualifiers:
                                ## end time
                                if qualifier_property == "P582":
                                    end_time = qualifier.getTarget().toTimestr()
                                ## start time    
                                if qualifier_property == "P580":
                                    start_time = qualifier.getTarget().toTimestr()
                                if qualifier_property == "P585":
                                    point_in_time = qualifier.getTarget().toTimestr()

                            #   info.append((item_qid, claim_lookup, targetid, targetstr, start_time, end_time, point_in_time))
                    info.append((item_qid, item_name, claim_lookup, claim_lookup_name, targetid, targetstr, start_time, end_time, point_in_time))
                    start_time = None
                    end_time = None
                    point_in_time  = None
                    #print(info)
                
            except:
                # claim_lookup == None
                # claim_lookup_name == None
                # targetid = None
                #  targetstr = None
                #  info.append((item_qid, item_name, claim_lookup, claim_lookup_name, targetid, targetstr, start_time, end_time, point_in_time))
                #  print("nay")
                pass
    except:
        pass
            
    return info

In [ ]:
# Test to see if it works 
r = getchangeinfo(site, 'Q25936435', [("P1546", "motto"), ("P1451", "motto text")])

for i in r:
    print(i)

In [ ]:
# Main part: This is where you read in the file with the ids and names to get other information 
# about the entities 
# If you want, you can also split up the file into smaller files to run parallel queries or keep track more easily,
# hence, the glob option
# Grab a cup of tea, this will take a while 
files = glob.glob("data/businesses-enterprises-companies.tsv")
df1 = pd.DataFrame(columns=['item_qid', 'item_name', 'claim_lookup', 'claim_lookup_name', 'targetid', 'targetstr', 'start_time', 'end_time', 'point_in_time'])

for file in files:
    print(file)
    time.sleep(0.7) # be nice 
    start = datetime.datetime.now()
    out_tsv = file + '_info.tsv'
    print(file, out_tsv)
    df1.to_csv(out_tsv, index=False,header=True,mode='a', sep='\t')
    input_list = pd.read_csv(file, sep='\t', names=["company"]).drop_duplicates()
   

    ## Things to query:
    # P1546 motto 
    # P1451 motto_text 
    # P1448 name 
    # P452 industry 
    # P1830 owner_of 
    # P355 subsidiary 
    # P169 ceo 
    # P1128 number_of_employees 
    # P1813 acronym 

    for index, row in input_list.iterrows():
        # print some info to know where we're at 
        print(row['company'])
        if '"' in row['company']:
            continue
        r = getchangeinfo(site, row['company'],[("P1546", "motto"), ("P1451", "motto text"), 
                                           ("P1448","name"), ("P452","industry"), ("P1830","owner of"),
                                           ("P355","subsidiary"), ("P169","ceo"), ("P1128","number of employees"),
                                           ("P1813","acronym")])
        df2 = pd.DataFrame(list(r), columns=['item_qid', 'item_name', 'claim_lookup', 'claim_lookup_name', 'targetid', 'targetstr', 'start_time', 'end_time', 'point_in_time'])
        df2.to_csv(out_tsv, index=False,header=False,mode='a', sep='\t')
    
    end = datetime.datetime.now() 
    print("start: " +  str(start) +  " end: "+ str(end))

We need a bit more information, namely the industry of the subsidiaries 

For this, you need to load in the file you just created, pull out the subsidiary ids and query those again 

In [ ]:
# Load the file into a dataframe 
files = glob.glob('data/businesses-enterprises-companies.tsv_info.tsv') #yes the name is somewhat ugly 

df = []
li = []

for filename in files:
    print(filename)
    temp_frame = pd.read_csv(filename, index_col=None, header=0, sep="\t")
    li.append(temp_frame)
    
df = pd.concat(li, axis=0, ignore_index=True)

df

In [ ]:
# Only select the subsidiary rows 
sub_data = df.loc[df['claim_lookup'].isin(['P1830', 'P355'])].drop_duplicates() 
sub_data['occurrences'] = sub_data.groupby('item_qid')['item_qid'].transform('size')
sub_data

In [ ]:
# Write to a tsv 
sub_data.to_csv('subsidiaries_company_enterprise_business.tsv', index=False,header=False,mode='a', sep='\t')

In [ ]:
# Query the data for the subsidiaries' industries 
# Grab a cup of tea, this will take a while 
files = glob.glob("subsidiaries_company_enterprise_business.tsv")
df1 = pd.DataFrame(columns=['item_qid', 'item_name', 'claim_lookup', 'claim_lookup_name', 'targetid', 'targetstr', 'start_time', 'end_time', 'point_in_time'])
x = 1 

for file in files:
    print(file)
    time.sleep(0.1)
    start = datetime.datetime.now()
    out_tsv = file + '_final_info.tsv'
    print(file, out_tsv)
    df1.to_csv(out_tsv, index=False,header=True,mode='a', sep='\t')
    #input_list = pd.read_csv(file, sep='\t', names=["company","companyLabel"]).drop_duplicates()
    input_list = pd.read_csv(file, sep='\t', names=["company"]).drop_duplicates()
    #input_list['company'] = input_list['company'].str.replace("http://www.wikidata.org/entity/","")

    for index, row in input_list.iterrows():
        print(x, row['company'])
        x = x+1 
        r = getchangeinfo(site, row['company'],[("P452","industry"), ("P1448", "name")])
        df2 = pd.DataFrame(list(r), columns=['item_qid', 'item_name', 'claim_lookup', 'claim_lookup_name', 'targetid', 'targetstr', 'start_time', 'end_time', 'point_in_time'])
        df2.to_csv(out_tsv, index=False,header=False,mode='a', sep='\t')
    
    end = datetime.datetime.now() 
    print("start: " +  str(start) +  " end: "+ str(end))